**Module 1**

In [ ]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import cv2
import networkx as nx
import tensorflow as tf
from google.colab import drive

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Configure TensorFlow for deterministic operations
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Mount Google Drive
drive.mount('/content/drive')

# Function to load and preprocess images
def load_data(data_dir, image_size=(64, 64)):
    images = []
    labels = []
    for label_folder in sorted(os.listdir(data_dir)):
        folder_path = os.path.join(data_dir, label_folder)
        if os.path.isdir(folder_path):
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                image = cv2.resize(image, image_size)
                images.append(image)
                labels.append(label_folder)
    return np.array(images), np.array(labels)

# Load dataset
data_dir = "/content/drive/My Drive/SwedishLeaf"
if not os.path.exists(data_dir):
    raise FileNotFoundError(f"The specified directory {data_dir} does not exist. Please check the path.")

images, labels = load_data(data_dir)

# Normalize images and encode labels
images = images / 255.0
images = images[..., np.newaxis]  # Add channel dimension
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(labels_encoded, num_classes=10)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=SEED)

# Build simplified ANN model
model = Sequential([
    Flatten(input_shape=X_train.shape[1:]),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_true, y_pred_classes) * 100
print(f"Test Accuracy: {accuracy:.2f}%")

# Function to predict a random image
def predict_random_image(model, images, labels, label_encoder):
    random_index = random.sample(range(len(images)), 1)[0]
    image = images[random_index]
    true_label = labels[random_index]
    image = image.reshape(1, image.shape[0], image.shape[1], image.shape[2])
    prediction = model.predict(image)
    predicted_class_index = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_class_index])[0]

    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"True: {true_label}, Predicted: {predicted_label}")
    plt.show()

    return image, true_label, predicted_label

# Example usage
image, true_label, predicted_label = predict_random_image(model, images, labels, label_encoder)
print(f"True Label: {true_label}")
print(f"Predicted Label: {predicted_label}")

# Plot training accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

# Plot simplified ANN architecture
import networkx as nx
import matplotlib.pyplot as plt

def plot_neural_network_with_non_overlapping_weights(input_size, hidden_size, output_size, title="Neural Network with Clear Weight Annotations"):
    """
    Plots a neural network diagram with symbolic weights (e.g., W[i,j] and V[j,k]) annotated on
    edges, ensuring clear, non-overlapping labels.
    """
    G = nx.DiGraph()

    # Input layer
    for i in range(input_size):
        G.add_node(f"Input {i+1}", layer=0)

    # Hidden layer
    for h in range(hidden_size):
        G.add_node(f"Hidden {h+1}", layer=1)

    # Output layer
    for o in range(output_size):
        G.add_node(f"Output {o+1}", layer=2)

    # Add edges with symbolic weights between layers
    edge_labels = {}
    for i in range(input_size):
        for h in range(hidden_size):
            edge_label = f"W[{i+1},{h+1}]"
            G.add_edge(f"Input {i+1}", f"Hidden {h+1}")
            edge_labels[(f"Input {i+1}", f"Hidden {h+1}")] = edge_label

    for h in range(hidden_size):
        for o in range(output_size):
            edge_label = f"V[{h+1},{o+1}]"
            G.add_edge(f"Hidden {h+1}", f"Output {o+1}")
            edge_labels[(f"Hidden {h+1}", f"Output {o+1}")] = edge_label

    # Define positions for a layered layout
    pos = {}
    layer_dist = 1.5  # distance between layers
    node_dist = 1  # distance between nodes

    # Input layer
    for i in range(input_size):
        pos[f"Input {i+1}"] = (0, i * node_dist - (input_size - 1) * node_dist / 2)

    # Hidden layer
    for h in range(hidden_size):
        pos[f"Hidden {h+1}"] = (layer_dist, h * node_dist - (hidden_size - 1) * node_dist / 2)

    # Output layer
    for o in range(output_size):
        pos[f"Output {o+1}"] = (2 * layer_dist, o * node_dist - (output_size - 1) * node_dist / 2)

    # Plot the network
    plt.figure(figsize=(12, 8))
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color="lightblue",
        edge_color="gray",
        node_size=2000,
        font_size=10,
    )

    # Draw edge labels for symbolic weights with adjusted positioning to avoid overlap
    nx.draw_networkx_edge_labels(
        G,
        pos,
        edge_labels=edge_labels,
        font_size=8,
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="white", alpha=0.7),
        label_pos=0.5,  # Place labels at the midpoint of the edges
    )

    plt.title(title)
    plt.show()

# Plot the neural network with non-overlapping symbolic weights for the given model
plot_neural_network_with_non_overlapping_weights(
    input_size=1,  # Assuming input has 64 features after Flatten
    hidden_size=4,  # Single hidden layer with 64 neurons
    output_size=1,  # Output layer with 10 neurons for classification
    title="Neural Network for Given Model with Clear Weight Annotations"
)
